**Setup pre-requisites for making API calls to SureMDM**

In [ ]:
import requests
import json
import base64
import os

#SureMDM API URL of your account. Replace this with your account URL
url = "https://your-domain.suremdm.io/api/"
#Add headers
headers = {
    #Api-Key header, Replace with your API KEY
    'ApiKey': "account-api-key-here",
    #Set content type
    'Content-Type': "application/json",
    }
#Basic authentication credentials
Credentials=("username","password")  #Replace with your username and password

# Account Details

**API Sample to Get My MDM Account Details**

In [ ]:
#Get Account details
response = requests.get(url+"account",auth=Credentials,headers=headers)
print(json.dumps(json.loads(response.text), indent=4))

# Jobs



## Get jobs

**API Sample to get Job List in my account**

In [ ]:
#Query parameters
params={"FolderId":""}
#Send request
response = requests.get(url+"job",auth=Credentials,params=params,headers=headers)
obj = json.loads(response.text)
print(json.dumps(json.loads(response.text), indent=4))


## Search job folders / Advance search on job grid


**API Sample to get all job folders / perform advance search on jobs in my account**

In [ ]:
Body=r"""{"FolderId":"",
"GroupId":"-1",
"SortColumn":"LastModified",
"SortOrder":"desc",
"Limit":50,
"Offset":0,
"SearchColumns":["Type"],
"AdanceSearch":true,
"AdvSearchValue":["folder"]}"""
#Send request
response = requests.post(url+"job/advancesearch",auth=Credentials,data=Body,headers=headers)
obj = json.loads(response.text)
print(json.dumps(json.loads(response.text), indent=4))

**API Sample to get all selective jobs / job folders in my account based on search value**
This api allows us to do the selective search based upon the attributes provided by the user. The feilds which defines this search are SearchColumns and AdvSearchValue.
AdvSearchValue supports CSV string too

**Note** : The index of search value and corresponding search columns should match
The search columns and thier description

*   Type - This defines the type of the object to the searched. It'll be a folder
*   Name - This defines the name of the object.

*   JobFolderPath - This defines the job location where it has been placed










In [ ]:
Body=r"""{"FolderId":"",
"GroupId":"-1",
"SortColumn":"LastModified",
"SortOrder":"desc",
"Limit":50,
"Offset":0,
"SearchColumns":["Type", "Name"],
"AdanceSearch":true,
"AdvSearchValue":["folder", "42GEARS"]}"""
#Send request
response = requests.post(url+"job/advancesearch",auth=Credentials,data=Body,headers=headers)
obj = json.loads(response.text)
print(json.dumps(json.loads(response.text), indent=4))

## Create Job


**API Sample to create jobs**

for more info: [API Doc](https://developer.42gears.com/suremdm/api/v1/#tag/JobsProfiles/paths/~1job/post)


In [ ]:
Payload = "{\"Subject\":\"Greetings!\",\"Body\":\"HappyBirthday!\",\"ReadNotification\":false,\"ForceRead\":true,\"EnableBuzz\":true,\"BuzzInterval\":\"12\",\"CloseDurationEnable\":true,\"Interval\":\"12\",\"RichTextBody\":\"\",\"RichTextHtml\":\"\"}"
base64PayLoad = base64.b64encode(bytes(Payload, 'utf-8')).decode('ascii')
Body = "{\"FolderId\":null,\"JobName\":\"Text Message Job\",\"JobType\":\"text_message\",\"Platform\":\"android\",\"PayLoad\":[\""+ base64PayLoad +"\"]}"

#Send request
response = requests.post(url+"job",auth=Credentials,data=Body,headers=headers)

#Api returns JobID and subjobID in CSV format. (JobID,SubjobID)
print(response.text)
jobId_text = response.text
# returns jobid

### Create Install Job/File Transfer Job
Creating these jobs includes 4 steps as follows.


In [ ]:
#Step 1: create jobid
Payload = "{ \"LocalPath\":\"vk3.apk\", \"DevicePath\":\"\/sdcard\/\", \"Install\":true,  \"UseBasicAuthentication\":false,  \"IsHttpUrl\":false,  \"HttpUserName\":\"\", \"HttpPassword\":\"\", \"IsSilent\":false,  \"ExecutePath\":\"\",  \"CurrentUser\":false, \"JobNameXML\":\"vk3.apk\",  \"FromAppStore\":false }"
base64PayLoad = base64.b64encode(bytes(Payload, 'utf-8')).decode('ascii')

#Multiple base64 payloads can be added in PayLoad array of below object to create job that installs multiple apps at once.
#Subsequent steps need to be repeated for each payload
Body = "{  \"FolderId\": null,  \"JobName\": \"Install Job astro2\",  \"JobType\": \"Install\",  \"Platform\": \"android\",  \"MinimumVersion\": \"0\",  \"VersionCompare\": \"2\",  \"PayLoad\": [ \""+ base64PayLoad +"\"  ]}"

#Send request
response = requests.post(url+"job",auth=Credentials,data=Body,headers=headers)

#Api returns JobID and subjobID in CSV format. (JobID,SubjobID)
print(response.text)
Ids = response.text.split(",")
jobId = Ids[0]
subJobId = Ids[1]

In [ ]:
#Step 2: get presigned S3 url to upload the file

#upload this file first if using Colab. Or your local file path
file_path = "/content/sample_data/vk3.apk"
filesize = os.path.getsize(file_path)
Body = "{\"JobID\":\""+ jobId +"\",\"SubJobID\":\""+ subJobId +"\",\"FileName\":\"vk3.apk\",\"FileSize\":"+ str(filesize) +"}"
print(Body)
#Send request
response = requests.post(url+"upload/UploadFile",auth=Credentials,data=Body,headers=headers)
print(response.text)
PreSignedS3URL = response.text

In [ ]:
#Step 3: Uploadfile to S3 url
#upload this file first if using Colab. Or your local file path
#files = { 'file': open(file_path, 'rb')}
#response = requests.put(PreSignedS3URL, data="", files=files)
#print(response.status_code)
# status code 200 means upload successful

# Open the file and get its contents
with open(file_path, "rb") as f:
    file_contents = f.read()

# Set the headers for the PUT request
uploadHeaders = {
    "Content-Type": "application/octet-stream",
    "Content-Length": str(len(file_contents))
}

# Send the PUT request with the file contents as the payload
response = requests.put(PreSignedS3URL, data=file_contents, headers=uploadHeaders)
print(response.status_code)

In [ ]:
# Check updated job size after upload is successful
response = requests.get(url+"job/"+ jobId +"/Size",auth=Credentials,headers=headers)
print(response.text)
#returns final size of the job

## Modify Job



**API sample to modify job**

In [ ]:
Payload = "{\"Subject\":\"Greetings!\",\"Body\":\"HappyBirthday To You!\",\"ReadNotification\":false,\"ForceRead\":true,\"EnableBuzz\":true,\"BuzzInterval\":\"12\",\"CloseDurationEnable\":true,\"Interval\":\"12\",\"RichTextBody\":\"\",\"RichTextHtml\":\"\"}"
base64PayLoad = base64.b64encode(bytes(Payload, 'utf-8')).decode('ascii')
Body = "{\"JobName\":\"TextMessageJobedited\",\"JobType\":\"TEXT_MESSAGE\",\"JobID\":\""+ jobId_text +"\",\"Platform\":\"ANY\",\"FolderId\":\"null\",\"PayLoad\":[\""+ base64PayLoad +"\"]}"

#Send request
response = requests.put(url+"job",auth=Credentials,data=Body,headers=headers)

#Api returns JobID and subjobID in CSV format. (JobID,SubjobID)
print(response.text)
jobId_text = response.text

### Modify Install/File Transfer Job

Steps are same as creating Install/File transfer jobs.
Only first steps is different. We have to use jobid of the job which we wish to edit in request body and HTTP method will be PUT.

In [ ]:
#Step 1: create jobid
Payload = "{ \"LocalPath\":\"vk3.apk\", \"DevicePath\":\"\/sdcard\/\", \"Install\":true,  \"UseBasicAuthentication\":false,  \"IsHttpUrl\":false,  \"HttpUserName\":\"\", \"HttpPassword\":\"\", \"IsSilent\":false,  \"ExecutePath\":\"\",  \"CurrentUser\":false, \"JobNameXML\":\"vk3.apk\",  \"FromAppStore\":false }"
base64PayLoad = base64.b64encode(bytes(Payload, 'utf-8')).decode('ascii')

#Multiple base64 payloads can be added in PayLoad array of below object to create job that installs multiple apps at once.
#Subsequent steps need to be repeated for each payload
Body = "{ \"JobID\":\""+ jobId +"\" ,  \"FolderId\": null,  \"JobName\": \"Install Job Modified\",  \"JobType\": \"Install\",  \"Platform\": \"android\",  \"MinimumVersion\": \"0\",  \"VersionCompare\": \"2\",  \"PayLoad\": [ \""+ base64PayLoad +"\"  ]}"

#Send request
response = requests.put(url+"job",auth=Credentials,data=Body,headers=headers)

#Api returns JobID and subjobID in CSV format. (JobID,SubjobID)
print(response.text)
Ids = response.text.split(",")
jobId = Ids[0]
subJobId = Ids[1]

In [ ]:
#Step 2: get presigned S3 url to upload the file


filesize = os.path.getsize(file_path)
Body = "{\"JobID\":\""+ jobId +"\",\"SubJobID\":\""+ subJobId +"\",\"FileName\":\"vk3.apk\",\"FileSize\":"+ str(filesize) +"}"
print(Body)
#Send request
response = requests.post(url+"upload/UploadFile",auth=Credentials,data=Body,headers=headers)
print(response.text)
PreSignedS3URL = response.text

In [ ]:
#Step 3: Uploadfile to S3 url
# files = { 'file': open("/content/sample_data/nixagent.apk", 'rb')}
# response = requests.put(PreSignedS3URL, data="", files=files)
# print(response.status_code)
# status code 200 means upload successful

# Open the file and get its contents
with open(file_path, "rb") as f:
    file_contents = f.read()

# Set the headers for the PUT request
uploadHeaders = {
    "Content-Type": "application/octet-stream",
    "Content-Length": str(len(file_contents))
}

# Send the PUT request with the file contents as the payload
response = requests.put(PreSignedS3URL, data=file_contents, headers=uploadHeaders)
print(response.status_code)

In [ ]:
# Check updated job size after upload is successful
response = requests.get(url+"job/"+ jobId +"/Size",auth=Credentials,headers=headers)
print(response.text)
#returns final size of the job

## Delete job

**API sample to delete jobs**

In [ ]:
Body = "{\"FolderId\":null,\"JobID\":\"[\\\""+ jobId +"\\\"]\",\"JobType\":\"JOBS\"}"

#Send request
response = requests.delete(url+"job",auth=Credentials,data=Body,headers=headers)
print(response.text)

# Device


for more info: [API Doc](https://developer.42gears.com/suremdm/api/v1/#tag/Device)

##List Devices

**API to List the Devices**

This api allows you to list all the devices on the basis of ID.

| ID      | Description               |
| -------          | -------------             |
| "AllDevices"     | To list all the devices present in console   |
| "null"           | To list all the devices from home group     |
|  ID of specific group or the tag | To list all the devices from specific group or tag. Eg:- "f9f40a04-033a-499c-9339-9cd6881e4c5c"

Note:- When tag ID provided in **ID** field, then **IsTag** must be true.

<details>
<summary><b style="font-size: 18px;">Click here to get more information regarding each field</b></summary>

| Permission ID              | Description                                          |
| -------------------------- | ---------------------------------------------------- |
| AdanceSearch               | Enable advanced search (true/false).Make it true in case of Advanced search otherwise keep it false.                 |
| AdvSearchJobID             | Advanced search job ID.                              |
| AdvSearchValue             | Values for advanced search.                           |
| SearchColumns              | Columns to be searched.                              |
| EnableDeviceGlobalSearch   | Enable global search for devices (true/false).       |
| ID                         | Group ID/Tag ID for filtering devices.                      |
| IsSearch                   | Indicate if it's a search operation (true/false).     |
| IsTag                      | Indicate if tag use for filteration (true/false). If tag ID used in ID field then true otherwise false   |
| Limit                      | Maximum number of results to retrieve.               |
| Offset                     | Number of results to skip before retrieving.         |
| SearchValue                | Search value for device search.                      |
| SortColumn                 | Column for sorting results.Sort the result on the basis of this value( "LastTimeStamp", "DeviceName" , "DeviceModelName", "PlatformType", "Battery", "AgentVersion", "DeviceTimeZone", "DeviceRegistered", "DataUsage")                          |
| SortOrder                  | Sort order (asc/desc) for sorting results.           |
| Columns                    | Columns to include in the response.                  |
| IsIncludedBlackListed      | Include blacklisted devices (true/false).            |

</details>


In [ ]:
ID = "null"      #Enter the ID
Body = f'''{{
    "AdanceSearch": false,
    "AdvSearchJobID": "",
    "AdvSearchValue": [],
    "SearchColumns": [],
    "EnableDeviceGlobalSearch": false,
    "ID":"{ID}",
    "IsSearch": false,
    "IsTag": false,
    "Limit": 2,
    "Offset": 0,
    "SearchValue": "",
    "SortColumn": "DeviceName",
    "SortOrder": "asc",
    "Columns": ["DeviceName","DeviceModelName","ConnectionStatus","AMTConnectionStatus","LastTimeStamp"],
    "IsIncludedBlackListed": false
}}'''

#Limit:maximum number of results (rows) you want to receive in the response
#Offset=0 =>start retrieving results from the beginning, Offset=10 =>skip the first 10 results and start retrieving results from the 11th one and so on


#Send request
response = requests.post(url+"devicegrid",auth=Credentials,data=Body,headers=headers)

print(json.dumps(json.loads(response.text), indent=4))

##Search Device (Basic Search)

**API to Search the Devices**

This API allows you to search all the devices based on ID and search value.

| ID                             | Description                                              |
| ------------------------------ | -------------------------------------------------------- |
| "AllDevices"                   | To search the devices present in the console             |
| "null"                         | To search the devices in the home group                  |
| ID of a specific group or tag   | To search the devices in a specific group or tag. Eg:- "f9f40a04-033a-499c-9339-9cd6881e4c5c"

| EnableDeviceGlobalSearch       | Description                                              |
| ------------------------------ | -------------------------------------------------------- |
| true                           | It will search among all the devices present in the console irrespective of ID.   |
| false                          | It will search according to the ID provided               |


<details>
<summary><b style="font-size: 18px;">Click here to get more information regarding each field</b></summary>

| Permission ID              | Description                                          |
| -------------------------- | ---------------------------------------------------- |
| AdanceSearch               | Enable advanced search (true/false).                 |
| AdvSearchJobID             | Advanced search job ID.                              |
| AdvSearchValue             | Values for advanced search.                          |
| SearchColumns              | Columns to be searched.                              |
| EnableDeviceGlobalSearch   | Enable global search for devices (true/false).       |
| ID                         | Group ID/Tag ID for filtering devices.                      |
| IsSearch                   | Indicate if it's a search operation (true/false).If you want to search keep it true otherwise false.    |
| IsTag                      | Indicate if tag use for filteration (true/false). If tag ID used in ID field then true otherwise false   |
| Limit                      | Maximum number of results to retrieve.               |
| Offset                     | Number of results to skip before retrieving.         |
| SearchValue                | Search value for device search.                      |
| SortColumn                 | Column for sorting results.Sort the result on the basis of this value( "LastTimeStamp", "DeviceName" , "DeviceModelName", "PlatformType", "Battery", "AgentVersion", "DeviceTimeZone", "DeviceRegistered", "DataUsage")                          |
| SortOrder                  | Sort order (asc/desc) for sorting results.           |
| Columns                    | Columns to include in the response.                  |
| IsIncludedBlackListed      | Include blacklisted devices (true/false).            |

</details>


In [ ]:
ID = "null"   #Enter the ID
SearchValue = "Test"     #Enter the value to be search
Body = f'''{{
    "AdanceSearch": false,
    "AdvSearchJobID": "",
    "AdvSearchValue": [],
    "SearchColumns": ["DeviceName","DeviceModelName","PlatformType","Operator","IMEI","Notes","DeviceUserName","DeviceID"],
    "EnableDeviceGlobalSearch": false,
    "ID":"{ID}",
    "IsSearch": true,
    "IsTag": false,
    "Limit": 2,
    "Offset": 0,
    "SearchValue": "{SearchValue}",
    "SortColumn": "DeviceName",
    "SortOrder": "asc",
    "Columns": ["DeviceName","DeviceModelName","ConnectionStatus","AMTConnectionStatus","LastTimeStamp"],
    "IsIncludedBlackListed": false
}}'''

#Limit:maximum number of results (rows) you want to receive in the response
#Offset=0 =>start retrieving results from the beginning, Offset=10 =>skip the first 10 results and start retrieving results from the 11th one and so on


#Send request
response = requests.post(url+"devicegrid",auth=Credentials,data=Body,headers=headers)

print(json.dumps(json.loads(response.text), indent=4))

##Search Device (Advanced search)

**API to Search the Devices**

This API allows you to search all the devices based on ID, SearchColumns and AdvSearchValue.

| ID                             | Description                                              |
| ------------------------------ | -------------------------------------------------------- |
| "AllDevices"                   | To search the devices present in the console             |
| "null"                         | To search the devices in the home group                  |
| ID of a specific group or tag   | To search the devices in a specific group or tag. Eg:- "f9f40a04-033a-499c-9339-9cd6881e4c5c"

| EnableDeviceGlobalSearch       | Description                                              |
| ------------------------------ | -------------------------------------------------------- |
| true                           | It will search among all the devices present in the console irrespective of ID.   |
| false                          | It will search according to the ID provided               |


<details>
<summary><b style="font-size: 18px;">Click here to get more information regarding each field</b></summary>

| Permission ID              | Description                                          |
| -------------------------- | ---------------------------------------------------- |
| AdanceSearch               | Enable advanced search (true/false).                 |
| AdvSearchJobID             | Advanced search job ID.                              |
| AdvSearchValue             | Values for advanced search. Add value of respective column that is added in SearchColumns to be search. For eg. ["1","2","Test"]                          |
| SearchColumns              | Columns to be searched. Add column to be search. value should be comma sepearted. For eg. ["ConnectionStatus","JobsFailed","DeviceName"]                             |
| EnableDeviceGlobalSearch   | Enable global search for devices (true/false). Make it true if you want to search among all devices present in console irrespective of ID otherwise keep it false.|
| ID                         | Group ID/Tag ID for filtering devices.                      |
| IsSearch                   | Indicate if it's a search operation (true/false).     |
| IsTag                      | Indicate if tag use for filteration (true/false). If tag ID used in ID field then true otherwise false   |
| Limit                      | Maximum number of results to retrieve.               |
| Offset                     | Number of results to skip before retrieving.         |
| SearchValue                | Search value for device search.                      |
| SortColumn                 | Column for sorting results.Sort the result on the basis of this value( "LastTimeStamp", "DeviceName" , "DeviceModelName", "PlatformType", "Battery", "AgentVersion", "DeviceTimeZone", "DeviceRegistered", "DataUsage")                          |
| SortOrder                  | Sort order (asc/desc) for sorting results.           |
| Columns                    | Columns to include in the response.                  |
| IsIncludedBlackListed      | Include blacklisted devices (true/false).            |


|  JobsFailed  |     Description    |
|-------------|---------------------|
|   "0"   |   Failed   |
|   "1"   |   Success   |
|   "2"   |   Pending   |
|   "3"   |   Success   |


|  ConnectionStatus  |     Description    |
|-------------|---------------------|
|   "0"   |   Offline   |
|   "1"   |   Online   |


</details>


In [ ]:
ID = "null"   #Enter the ID
Body = f'''{{
    "AdanceSearch": true,
    "AdvSearchJobID": "",
    "AdvSearchValue": [
        "1",
        "2",
        "T"
    ],
    "SearchColumns": [
        "ConnectionStatus",
        "JobsFailed",
        "DeviceName"
    ],
    "EnableDeviceGlobalSearch": false,
    "ID": "{ID}",
    "IsSearch": true,
    "IsTag": false,
    "Limit": 1000,
    "Offset": 0,
    "SearchValue": "",
    "SortColumn": "ConnectionStatus",
    "SortOrder": "desc",
    "Columns": [
        "DeviceName",
        "DeviceModelName",
        "ConnectionStatus"
    ],
    "IsIncludedBlackListed": false,
}}'''

#Limit:maximum number of results (rows) you want to receive in the response
#Offset=0 =>start retrieving results from the beginning, Offset=10 =>skip the first 10 results and start retrieving results from the 11th one and so on


#Send request
response = requests.post(url+"devicegrid",auth=Credentials,data=Body,headers=headers)

print(json.dumps(json.loads(response.text), indent=4))

##Apply Job On Device

**API sample to apply job on the device**

In [ ]:
#Replace DeviceId and JobID with your values below
Body = r"""{
   "DeviceIds":[
      "8a56cb82-faae-4856-b66f-3218c28a8e2e"
   ],
   "JobId":"d04c0919-f475-4dbc-892f-63f892ab5031",
   "DownloadType":"2",
   "DeviceChargingState":"0"
}"""

#Send request
response = requests.post(url+"jobassignment",auth=Credentials,data=Body,headers=headers)

print(response.text)
jobId = response.text

##Delete Device(s)

**API sample to delete device(s)**

In [ ]:
# DeviceId=> Multiple DeviceIds can be passed and value can be comma seperated
DeviceId= "c16d54d2-0129-4d8e-926e-6a2f19c84b8f,79c2b437-5e7b-4f27-ad7a-49c8961a51bb"
deviceid_list = DeviceId.split(',')

if deviceid_list is not None and len(deviceid_list) > 0:
  DeviceCount = len(deviceid_list)
  Body = f'''
  {{
      "Action": "FORCEDELETE_DEVICE",
      "DeviceId": "{DeviceId}",
      "VerificationMsg": "delete-{DeviceCount}"
  }}
  '''


  #Send request
  response = requests.put(url+"device/delete",auth=Credentials,data=Body,headers=headers)

  if response.status_code == 200:
    response_data = json.loads(response.text)
    if response_data is not None and 'status' in response_data and response_data["status"] :
      print("Device(s) successfully deleted")
    else:
      print("Device(s) could not be deleted")

# Group


## List Groups

**API sample to list all the groups**

In [ ]:
#Send request
response = requests.get(url+"group/1/getAll",auth=Credentials,headers=headers)
print(json.dumps(json.loads(response.text), indent=4))


## List Groups based on pagination and sort

**API sample to list all the groups based on pagination and sort**

This api allows to list all the groups with pagination and sort. The route of the endpoint is url/group/getAll/limit/offset/sortColumn/sortOrder

The sort columns supported are : GroupID and GroupName

In [ ]:
#Send request
response = requests.get(url+"group/getAll/10/0/GroupName/asc",auth=Credentials,headers=headers)
print(json.dumps(json.loads(response.text), indent=4))

In [ ]:
Body = r"""{
    "SearchValue": [
        "!!proxy"
    ],
    "Column": "GroupName"
}"""

#Send request
response = requests.post(url+"v2/Group/Search",auth=Credentials,data=Body,headers=headers)
print(json.dumps(json.loads(response.text), indent=4))

## Create Group
**API sample to create a group**

In [ ]:
Body = r"""{
   "GroupID":null,
   "GroupName":"colab group"
}"""
#GroupID: ID of the parent group, null means HOME

#Send request
response = requests.post(url+"group",auth=Credentials,data=Body,headers=headers)

print(response.text)
GroupID = response.text
# returns GroupID of the newly created group

## Apply Job On Group
**API sample to apply job on particular group**

For more info: [API Doc](https://developer.42gears.com/suremdm/api/v1/#tag/Group/paths/~1jobassignment~1group/post)

In [ ]:
#Replace GroupID and JobID with your values below
Body = r"""{
   "JobId":"d04c0919-f475-4dbc-892f-63f892ab5031",
   "ServerScheduleTime":null,
   "RecursiveScheduleTime":0,
   "DownloadType":"2",
   "GroupID":"8851b4d9-c409-4f4d-a3e2-d9a1a68f4079"
}"""

#Send request
response = requests.post(url+"jobassignment/group/",auth=Credentials,data=Body,headers=headers)

print(response.text)

## Get Default Jobs On Group


In [ ]:
#Get default jobs for a group

response = requests.get(url+"defaultjob/36e3d555-3522-4bea-b360-123b7e5bebe2",auth=Credentials,headers=headers)
print(json.dumps(json.loads(response.text), indent=4))

## Assign Default Jobs On Group

**API sample to assign default jobs on group**

In [ ]:
Body = r"""[
  {
    "JobID": "d04c0919-f475-4dbc-892f-63f892ab5031",
    "GroupID": "8851b4d9-c409-4f4d-a3e2-d9a1a68f4079",
    "ForceApply": false,
    "DownloadType": "2",
    "DeviceChargingState": "0"
  }
]"""
#DownloadType = > 0:wifi, 1: mobile data, 2: Any
#DeviceChargingState = > 0:don't care, 1: plugged in
#GroupID =>  null for home group

#Send request
response = requests.post(url+"defaultjob/",auth=Credentials,data=Body,headers=headers)

print(response.text)

## Unassign Default Jobs from the Group


In [ ]:
#Delete Default Job
import urllib.parse
JobIds = urllib.parse.quote("""["d04c0919-f475-4dbc-892f-63f892ab5031"]""")

#Send request
response = requests.delete(url+"defaultjob/"+JobIds+"/8851b4d9-c409-4f4d-a3e2-d9a1a68f4079",auth=Credentials,headers=headers)
print(response.text)

#Tags

## Apply default job on tag

**API sample to apply default job on tag**

In [ ]:
Body = r"""[
  {
    "JobID": "d04c0919-f475-4dbc-892f-63f892ab5031",
    "GroupID": "cd2cf195-ee66-45e6-b17d-ae7b93036135",
    "ForceApply": false,
    "DownloadType": "2",
    "DeviceChargingState": "0"
  }
]"""
#DownloadType = > 0:wifi, 1: mobile data, 2: Any
#DeviceChargingState = > 0:don't care, 1: plugged in
#GroupID (In this case tagid) =>  null for home group

#Send request
response = requests.post(url+"tagsdefaultjobs",auth=Credentials,data=Body,headers=headers)

print(response.text)

##Apply job on tag

**API sample to apply job on tag**

For more info: [API Doc](https://developer.42gears.com/suremdm/api/v1/#tag/Tags/paths/~1jobassignment~1tags/post)

In [ ]:
Body = r"""{
  "JobId": "d04c0919-f475-4dbc-892f-63f892ab5031",
  "ServerScheduleTime": null,
  "RecursiveScheduleTime": 0,
  "DownloadType": "2",
  "DeviceChargingState": "0",
  "TagId": "cd2cf195-ee66-45e6-b17d-ae7b93036135"
}"""
#DownloadType = > 0:wifi, 1: mobile data, 2: Any
#DeviceChargingState = > 0:don't care, 1: plugged in

#Send request
response = requests.post(url+"jobassignment/tags/",auth=Credentials,data=Body,headers=headers)

print(response.text)

# Filter

## Get filters

**API sample to list all the filters**

In [ ]:
#Send request
response = requests.get(url+"devicefilter",auth=Credentials,headers=headers)
print(json.dumps(json.loads(response.text), indent=4))
Filters = json.loads(response.text)

## Get devices from filter

**API sample to apply filter and get devices**

uses filter from above API's result

In [ ]:
filter = Filters[0]
filterQuery = json.loads(filter["FilterQuery"])
print(filter["FilterName"])
SearchCols = list(filterQuery.keys())
SearchVals = list(filterQuery.values())

payload = r"""{
  "SearchColumns": """ + str(SearchCols) + """,
  "AdvSearchValue": """ + str(SearchVals) + """,
  "ModelSearchOption": "0",
  "NameSearchOption": "0",
  "ID": null,
  "IsTag": false,
  "SortColumn": "LastTimeStamp",
  "SortOrder": "desc",
  "Limit": 10,
  "Offset": 0,
  "IsSearch": true,
  "IsIncludedBlackListed": false,
  "AdanceSearch": true,
  "EnableDeviceGlobalSearch": false,
  "SearchValue": ""
}"""
print(payload)
#Send request
response = requests.post(url+"devicegrid",auth=Credentials,data=payload,headers=headers)
print(response.text)


## Apply job on filter

**API sample to apply job on filter devices**

In [ ]:
payload = r"""{
"FilterID":"10",
"JobId":"cb4f5e0c-1f2b-450f-95c6-84e1efbf72e1",
"DeviceChargingState": "0",
"DownloadType": "2"
}"""

#Send request
response = requests.post(url+"jobassignment/deviceFilter",auth=Credentials,data=payload,headers=headers)
print(response.text)

# Job Queue

## Get Job Queue (Device)

**API sample to list all the pending jobs on the particular device**

In [ ]:
Body = r"""{
   "DeviceIds":"8a56cb82-faae-4856-b66f-3218c28a8e2e",
}"""

#Send request
response = requests.post(url+"jobqueue/10/0/false/null/Time/desc",auth=Credentials,data=Body,headers=headers)
#Endpoint: {limitJobQueue}/{offsetJobQueue}/{isSearch}/{searchValue}/{sortColumn}/{sortOrder}
#Note: Search is not supported by API so isSearch should be false
print(json.dumps(json.loads(response.text), indent=4))
ResponseObject = json.loads(response.text)

**API sample to list all the failed/deployed jobs on the particular device**

In [ ]:
#Send request
response = requests.get(url+"JobQueueErrorHistory/6ef89b84-c7e8-4960-a863-92ebe0660fa3/false/10/0/false/null/Time/des",auth=Credentials,headers=headers)
#Endpoint: JobQueueErrorHistory/{deviceId}/{isSuccess}/{limitJobQueue}/{offsetJobQueue}/{isSearch}/{searchValue}/{sortColumn}/{sortOrder}
#Note: Search is not supported by API so isSearch should be false
print(json.dumps(json.loads(response.text), indent=4))
ResponseObject = json.loads(response.text)

##Delete Job From Job Queue (Device)

In [ ]:
import urllib.parse
JobIds = urllib.parse.quote("""["0438f622-2f51-49d1-b675-d67d7b4e7114"]""")
DeviceIds = urllib.parse.quote("[\"6ef89b84-c7e8-4960-a863-92ebe0660fa3\"]")
RowIds = urllib.parse.quote("[\"bce12067-dbe7-44a7-a79b-a24e08b9b564\"]")
print(url+"jobqueue/"+JobIds+"/"+DeviceIds+"/"+RowIds)
#Send request
response = requests.delete(url+"jobqueue/"+JobIds+"/"+DeviceIds+"/"+RowIds,auth=Credentials,headers=headers)
print(response.text)

## Re-Apply Job in Jobqueue (Device)

**Re-Apply Failed/Inprogress jobs from jobqueue**

In [ ]:
Body = r"""{
   "JobID":"607405d4-3128-4dc5-9c2f-acf5cd24c51d",
   "DeviceId":"6ef89b84-c7e8-4960-a863-92ebe0660fa3",
   "RowId":"6949843d-bce9-4222-bde5-caac43f5f27a",
   "Status":"Error",
   "JobPlatform":"ANDROID"
}"""
#RowId is jobqueu id

#Send request
response = requests.put(url+"jobqueue",auth=Credentials,data=Body,headers=headers)
print(response.text)

## Get Job Queue (Group)

In [ ]:
#Send request
response = requests.get(url+"GroupJobQueue/8851b4d9-c409-4f4d-a3e2-d9a1a68f4079/1",auth=Credentials,headers=headers)
#Endpoint: GroupJobQueue/{GroupId}/{Type}
#Type-> 0: Pending to deploy jobs, 1: Succefully Deployed jobs, 2: deployment In-Progress jobs, 3: Failed to deploy jobs
print(response.text)

##Delete Job From Job Queue (Group)

In [ ]:
#Send request
response = requests.get(url+"GroupJobQueue/0438f622-2f51-49d1-b675-d67d7b4e7114/5419057d-bcb7-4e15-a9d3-3c4d801d0276/0",auth=Credentials,headers=headers)
#Endpoint: GroupJobQueue/{JobId}/{GroupId}/{Type}
#Type-> 0: Pending to deploy jobs, 1: Succefully Deployed jobs, 2: deployment In-Progress jobs, 3: Failed to deploy jobs
print(response.text)


##Re-Apply Job in Jobqueue (Group)

In [ ]:
#Send request
response = requests.put(url+"GroupJobQueue/0438f622-2f51-49d1-b675-d67d7b4e7114/5419057d-bcb7-4e15-a9d3-3c4d801d0276/0",auth=Credentials,headers=headers)
#Endpoint: GroupJobQueue/{JobId}/{GroupId}/{Type}
#Type-> 0: Pending to deploy jobs, 1: Succefully Deployed jobs, 2: deployment In-Progress jobs, 3: Failed to deploy jobs
print(response.text)

# App Store



## List Apps

**API sample to list all the Apps from app store**



In [ ]:
#Send request
response = requests.get(url+"EAM/Android/false",auth=Credentials,headers=headers)
#endpoint: EAM/{Platform}/{WithIcon}
#Platform -> Android, ios, macOS, Windows, Linux

print(json.dumps(json.loads(response.text), indent=4))

## Upload App

In [ ]:


# Replace <file_path> with the path to the file you want to upload
file_path = "/content/sample_data/vk3.apk"
# Replace <file_name> with the name of the file you want to upload
file_name = "vk3.apk"

appsize = os.path.getsize(file_path)
Body = f"""{{
   "AppTitle":"{str(file_name)}",
   "AppSize":"{str(appsize)}",
   "IsS3Storage":true,
   "PlatformType":"Android"
}}"""

print(Body)
#Send request
response = requests.post(url+"EAM",auth=Credentials,data=Body,headers=headers)
print(response.text)
ResponseObjectEAM = json.loads(response.text)

In [ ]:
# Open the file and get its contents
with open(file_path, "rb") as f:
    file_contents = f.read()

# Set the headers for the PUT request
uploadHeaders = {
    "Content-Type": "application/octet-stream",
    "Content-Length": str(len(file_contents))
}

# Send the PUT request with the file contents as the payload
response = requests.put(str(ResponseObjectEAM["SignedURL"]), data=file_contents, headers=uploadHeaders)
print(response.status_code)

In [ ]:
Body = f"""{{
    "GUID": "{str(ResponseObjectEAM["GUID"])}",
    "AppURL": "{str(ResponseObjectEAM["AppURL"])}",
    "IsS3Storage": true,
    "PlatformType": "Android"
}}"""
print(Body)
#Send request
response = requests.put(url+"EAMGetAppDetails",auth=Credentials,data=Body,headers=headers)
print(response.text)
ResponseObject = json.loads(response.text)

In [ ]:
ResponseObject["AppCategory"] = "Test app"
ResponseObject["AppDescription"] = "This is test app"
ResponseObject["AppTitle"] = "app titie"
ResponseObject["AppSize"] = str(appsize)

Body = json.dumps(ResponseObject)
print(Body)
#Send request
response = requests.post(url+"EAMSaveAppDetails",auth=Credentials,data=Body,headers=headers)
print(response.text)

# Profiles


## List Profiles

**API sample to list profiles**
For more info: [API Doc](https://developer.42gears.com/suremdm/api/v1/#tag/JobsProfiles/paths/~1profile~1List~1{platform}~1{profileFolderId}/get)

In [ ]:
#Send request
response = requests.get(url+"Profile/List/ANDROID/null/10/0/LastModified/desc",auth=Credentials,headers=headers)
#Endpoint = Profile/List/{platform}/{profileFolderId}/{limit}/{offSet}/{sortColumn}/{sortOrder}
print(response.text)

## Create Profile

**Sample API to Create a Profile**

In this example we configure *Application Policy*(using app store) and *Password Policy*

For more info: refer ***Create Profile*** option [API Doc](https://developer.42gears.com/suremdm/api/v1/#tag/JobsProfiles/paths/~1job/post)

In [ ]:
Payload = """{
  \\\"applicationPolicy\\\":{
      \\\"enablePlayStore\\\":false
      },
  \\\"enterpriseAppStore\\\":{
	  \\\"eamApps\\\":[
		  {\\\"AppTitle\\\":\\\"DeviceInfo\\\",
			\\\"AppVersion\\\":\\\"1.0.1\\\",
			\\\"AutoInstallApp\\\":true,
			\\\"PlatformType\\\":\\\"Android\\\",
			\\\"AppCategory\\\":\\\"11\\\",
			\\\"AppDescription\\\":\\\"11\\\",
			\\\"AppIcon\\\":\\\"iVBORw0KGgoAAAANSUhEUgAAADAAAAAwCAQAAAD9CzEMAAAE3klEQVRYw6WYW48URRTHf1Xd07Mzy3JZNEGXjQqMgAS5xBB0ExLURMODL0aNRL+DT34K\\\/RA+6JMkoFE00egDunHJohIDiAgubiCwctndZqZvVeXD1Mz0bZpltzqZTFVXn8v\\\/\\\/M\\\/pUy0M6SEE7qamVzeCVYwgXuoQGp1eczPina0bHp8aPU7LOKuQb7iTfP3vCXHTxCmZJmX9xPi2j9YfX\\\/JiVjckY0rPXTl27YpRZQq8Q29s\\\/jT2xlj9COkY5\\\/TM+4t3SyDa2Gy81\\\/F2IFjLuCrqU6NjlCmojbADJGsbDtJz3dIgG4GzdgWCLD+yLEIMVSDs\\\/S5dDKYi1EMVFG\\\/3hEtcXFwEAo0iQZFAiRpZrUAWQixx8GhSY5wmDRwCOtwhJKBDgi6JwlAFpgCRpMY6NtDiIGn6JpznEgv4BOiMHxJBuDKIBJIGo0xwjHoODocDHOAMF1jmAUnqnshh4JYFs3drlA0cZo8VblAoDA6ONWOKnXzLAg+IVsYiUhBJmmzibRp9lBNuM4tiK3sYsYaM8y4nmWeZpMTEyhjUaTLFSN95TchlTgPbmaSWsvMYn5HQtrFQubC7efulxXgdk2xLYTvgvrGA9dYE+zjJPUIMcJlJ\\\/dA8EDRo8ErBlmd4Gc1WGjYavbGbc8QoVAbk0rwSCAQONV5AWgtN3\\\/rNHOEltiNIUOjU9SYN6jhI5PAgtzGWxzUatHJZqom4xnckPMuLjOYsqzFh005UZfKiiojw8NjYh8dg0Bg0bS7yFTDHbkYK+TvJNQtvBU2X9T1uI9nIeqKCB4oAgJgYXSgpT6FwSmKQo6m0PPIKNcaUMmowxnFwrIRKFmkMEc2CAp2qObrEA4mwe5zhtcglQSOR3C9RkM6DooKAGGUVVCQaaCChU4AgDUsZRHfRqIexqOsmxPiPDNElnD5EFQokBk3Ef0S58AwUlEN0k9juqMjkrnhFRMQvuTwusih9LXGb0Ba6CgXCMkHR4TyLmWLQvQbeZK9TRER2h1PlAVZBQJsvC2J6EOXXp1lgmcTuqYRI2EdilllgJgODoGbrjpuBaYlztIls+WMlLOp2mT5n2cJE6j28m3dQPMm6FMdCTtCmg1pJXyRSCqCD4RQtjtpHJI9xBGMLcnffRaZZws+8HSppmu4dEtokXOQGR60fgzpjgJBvuGGtNyvrixxMJsG6lO3wBVt4mr0p5+f5g3k6+MS5lNQIW3VLi50uZLAioYPPHD8ziockJCAgISTK2d6ryJUQ6ZKTkSJE4HIXiUBhSFLF7xEUiBIP0u1iWsywoas90MSs5vw3GHG1gjp\\\/463hEKVQwzu7ti0DoV2M+x1DFyBTaOu7POuKESV7SmiqeYKWfcjjdwz7bWNb40\\\/m+zsPM2LLQsIZXiW2hXqRX3PETM+0SAyTzPQX9qKZ7s+eZ65viOCs\\\/b+TCea4YWf7URXFLlDnx2hTw8fHp8l9fJp2VqPdr54xHm18fB7Q5Bab7Z4AAdSCMC73oH31k0Nv\\\/dDcyTYA7vEb8ByT3Z6JmRTC0+xHAIZZfC5w0MbgJ15X\\\/3zO4pBPCaw\\\/9EHrw+sjS1JkzpW93+y\\\/\\\/F1B0+yLo9mPj3N98EFEpOMuJGNbWrtea+4Sbu9AKG2fIB4yl0iT3Prr+x9nWTBJqQe9Ly7UV5kKBkVAYjIi\\\/wehm0uZKZ3sSgAAAABJRU5ErkJggg==\\\",
			\\\"AppPackage\\\":\\\"com.curvefish.apps.deviceinfo\\\",
			\\\"AppSize\\\":\\\"82494\\\",
			\\\"AppURL\\\":\\\"062200038\\\/EAM\\\/Android\\\/5bdf9773-af0d-4745-ae9b-f066ea767b79\\\",
			\\\"GUID\\\":\\\"5bdf9773-af0d-4745-ae9b-f066ea767b79\\\",
			\\\"IsS3Storage\\\":\\\"true\\\",
			\\\"AppVersionCode\\\":\\\"2\\\",
			\\\"ApplicationType\\\":\\\"EnterpriseApplication\\\",
			\\\"AppType\\\":0
		  }
		]
	},
	\\\"passwordPolicy\\\":{
		\\\"ProfileType\\\":\\\"3\\\",
		\\\"workProfileQuality\\\":\\\"262144\\\",
		\\\"workProfileMinimumLength\\\":\\\"16\\\",
		\\\"workProfileMinimumLetterLength\\\":null,
		\\\"workProfileMinimumLowerCaseLength\\\":null,
		\\\"workProfileMinimumNonLetterLength\\\":null,
		\\\"workProfileMinimumNumericLength\\\":null,
		\\\"workProfileMinimumSymbolLength\\\":null,
		\\\"workProfileMinimumUpperCaseLength\\\":null,
		\\\"quality\\\":\\\"131072\\\",
		\\\"expirationTimeout\\\":3204000000,
		\\\"historyLength\\\":\\\"5\\\",
		\\\"maximumFailedAttempts\\\":\\\"3\\\",
		\\\"minimumLength\\\":\\\"7\\\",
		\\\"maximumTimeToLock\\\":699000
	},
}"""
Body = "{\"FolderId\":null,\"JobName\":\"Test Profile\",\"JobType\":\"ANDROID WORK PROFILE\",\"Platform\":\"Android\",\"PayLoad\":[\""+ Payload +"\"]}"
print(Body)
#Send request
response = requests.post(url+"job",auth=Credentials,data=Body,headers=headers)

#Api returns ProfileID
print(response.text)
jobId = response.text
# returns jobid

## Modify Profile

**Sample API for modifying Profile**

In this example we configure *Application Policy*(using app store) and *Password Policy*

For more info: refer ***Modify Profile*** option [API Doc](https://developer.42gears.com/suremdm/api/v1/#tag/JobsProfiles/paths/~1job/put)

In [ ]:
Payload = """{
  \\\"applicationPolicy\\\":{
      \\\"enablePlayStore\\\":false
      },
  \\\"enterpriseAppStore\\\":{
	  \\\"eamApps\\\":[
		  {\\\"AppTitle\\\":\\\"DeviceInfo\\\",
			\\\"AppVersion\\\":\\\"1.0.1\\\",
			\\\"AutoInstallApp\\\":true,
			\\\"PlatformType\\\":\\\"Android\\\",
			\\\"AppCategory\\\":\\\"11\\\",
			\\\"AppDescription\\\":\\\"11\\\",
			\\\"AppIcon\\\":\\\"iVBORw0KGgoAAAANSUhEUgAAADAAAAAwCAQAAAD9CzEMAAAE3klEQVRYw6WYW48URRTHf1Xd07Mzy3JZNEGXjQqMgAS5xBB0ExLURMODL0aNRL+DT34K\\\/RA+6JMkoFE00egDunHJohIDiAgubiCwctndZqZvVeXD1Mz0bZpltzqZTFVXn8v\\\/\\\/M\\\/pUy0M6SEE7qamVzeCVYwgXuoQGp1eczPina0bHp8aPU7LOKuQb7iTfP3vCXHTxCmZJmX9xPi2j9YfX\\\/JiVjckY0rPXTl27YpRZQq8Q29s\\\/jT2xlj9COkY5\\\/TM+4t3SyDa2Gy81\\\/F2IFjLuCrqU6NjlCmojbADJGsbDtJz3dIgG4GzdgWCLD+yLEIMVSDs\\\/S5dDKYi1EMVFG\\\/3hEtcXFwEAo0iQZFAiRpZrUAWQixx8GhSY5wmDRwCOtwhJKBDgi6JwlAFpgCRpMY6NtDiIGn6JpznEgv4BOiMHxJBuDKIBJIGo0xwjHoODocDHOAMF1jmAUnqnshh4JYFs3drlA0cZo8VblAoDA6ONWOKnXzLAg+IVsYiUhBJmmzibRp9lBNuM4tiK3sYsYaM8y4nmWeZpMTEyhjUaTLFSN95TchlTgPbmaSWsvMYn5HQtrFQubC7efulxXgdk2xLYTvgvrGA9dYE+zjJPUIMcJlJ\\\/dA8EDRo8ErBlmd4Gc1WGjYavbGbc8QoVAbk0rwSCAQONV5AWgtN3\\\/rNHOEltiNIUOjU9SYN6jhI5PAgtzGWxzUatHJZqom4xnckPMuLjOYsqzFh005UZfKiiojw8NjYh8dg0Bg0bS7yFTDHbkYK+TvJNQtvBU2X9T1uI9nIeqKCB4oAgJgYXSgpT6FwSmKQo6m0PPIKNcaUMmowxnFwrIRKFmkMEc2CAp2qObrEA4mwe5zhtcglQSOR3C9RkM6DooKAGGUVVCQaaCChU4AgDUsZRHfRqIexqOsmxPiPDNElnD5EFQokBk3Ef0S58AwUlEN0k9juqMjkrnhFRMQvuTwusih9LXGb0Ba6CgXCMkHR4TyLmWLQvQbeZK9TRER2h1PlAVZBQJsvC2J6EOXXp1lgmcTuqYRI2EdilllgJgODoGbrjpuBaYlztIls+WMlLOp2mT5n2cJE6j28m3dQPMm6FMdCTtCmg1pJXyRSCqCD4RQtjtpHJI9xBGMLcnffRaZZws+8HSppmu4dEtokXOQGR60fgzpjgJBvuGGtNyvrixxMJsG6lO3wBVt4mr0p5+f5g3k6+MS5lNQIW3VLi50uZLAioYPPHD8ziockJCAgISTK2d6ryJUQ6ZKTkSJE4HIXiUBhSFLF7xEUiBIP0u1iWsywoas90MSs5vw3GHG1gjp\\\/463hEKVQwzu7ti0DoV2M+x1DFyBTaOu7POuKESV7SmiqeYKWfcjjdwz7bWNb40\\\/m+zsPM2LLQsIZXiW2hXqRX3PETM+0SAyTzPQX9qKZ7s+eZ65viOCs\\\/b+TCea4YWf7URXFLlDnx2hTw8fHp8l9fJp2VqPdr54xHm18fB7Q5Bab7Z4AAdSCMC73oH31k0Nv\\\/dDcyTYA7vEb8ByT3Z6JmRTC0+xHAIZZfC5w0MbgJ15X\\\/3zO4pBPCaw\\\/9EHrw+sjS1JkzpW93+y\\\/\\\/F1B0+yLo9mPj3N98EFEpOMuJGNbWrtea+4Sbu9AKG2fIB4yl0iT3Prr+x9nWTBJqQe9Ly7UV5kKBkVAYjIi\\\/wehm0uZKZ3sSgAAAABJRU5ErkJggg==\\\",
			\\\"AppPackage\\\":\\\"com.curvefish.apps.deviceinfo\\\",
			\\\"AppSize\\\":\\\"82494\\\",
			\\\"AppURL\\\":\\\"062200038\\\/EAM\\\/Android\\\/5bdf9773-af0d-4745-ae9b-f066ea767b79\\\",
			\\\"GUID\\\":\\\"5bdf9773-af0d-4745-ae9b-f066ea767b79\\\",
			\\\"IsS3Storage\\\":\\\"true\\\",
			\\\"AppVersionCode\\\":\\\"2\\\",
			\\\"ApplicationType\\\":\\\"EnterpriseApplication\\\",
			\\\"AppType\\\":0
		  }
		]
	},
	\\\"passwordPolicy\\\":{
		\\\"ProfileType\\\":\\\"3\\\",
		\\\"workProfileQuality\\\":\\\"262144\\\",
		\\\"workProfileMinimumLength\\\":\\\"16\\\",
		\\\"workProfileMinimumLetterLength\\\":null,
		\\\"workProfileMinimumLowerCaseLength\\\":null,
		\\\"workProfileMinimumNonLetterLength\\\":null,
		\\\"workProfileMinimumNumericLength\\\":null,
		\\\"workProfileMinimumSymbolLength\\\":null,
		\\\"workProfileMinimumUpperCaseLength\\\":null,
		\\\"quality\\\":\\\"131072\\\",
		\\\"expirationTimeout\\\":3204000000,
		\\\"historyLength\\\":\\\"5\\\",
		\\\"maximumFailedAttempts\\\":\\\"3\\\",
		\\\"minimumLength\\\":\\\"7\\\",
		\\\"maximumTimeToLock\\\":699000
	},
}"""
Body = "{\"JobID\": \"d376835c-9c96-431d-9c28-ed5eff4130ca\",\"FolderId\":null,\"JobName\":\"Test Profile\",\"JobType\":\"ANDROID WORK PROFILE\",\"Platform\":\"Android\",\"PayLoad\":[\""+ Payload +"\"]}"
print(Body)
#Send request
response = requests.put(url+"job",auth=Credentials,data=Body,headers=headers)

#Api returns ProfileID
print(response.text)
ProfileID = response.text
# returns jobid

##Delete Profile

In [ ]:
Body = "{\"FolderId\":null,\"JobID\":\""+ ProfileID +"\",\"JobType\":\"ANDROID WORK PROFILE\",\"Platform\":\"ANDROID\"}"

#Send request
response = requests.delete(url+"job",auth=Credentials,data=Body,headers=headers)
print(response.text)

#PFW Apps

Get Pfw app deatils and Install on device

## Get PFW App Details

In [ ]:
#Send request
import json
response = requests.get(url+"afwapp/GetPFWAppDetails/com.google.android.gm",auth=Credentials,data="",headers=headers)

#print(response.text)
print(json.dumps(json.loads(response.text), indent=2))

## Install PFW App On Device


In [ ]:
#Replace GroupID and JobID with your values below
Body = r"""{
    ProductId:"net.sourceforge.opencamera",
    DeviceIds:["a975191a-8603-40ec-b34a-08eba69b2ad6", "gfsd"],
    AutoUpdateMode: "",
    InstallPolicy: {
		"InstallMode" : "forceAutoInstall",
		"InstallPriority" : 1,
		"InstallConstraint" : {
			"NetworkType" : "anyNetwork",
			"ChargingState" : "chargingNotRequired",
			"DeviceIdleState" : "deviceIdleNotRequired"
		}
	} ,
    Restrictions:[
        {
            "Title": "Email Address",
            "Description": "A variable field that supports a specific email address (janedoe@company.com) or templated value ($emailaddress$). EMMs can use this field to pull user credentials from Active Directory.",
            "Key": "email_address",
            "Value": "a@a.com",
            "Type": "string",
            "SelectKeys": "",
            "SelectValues": "",
            "NestedRestriction": null
        },

    ],
   "Permissions": [
    {
      "permission": "android.permission.ACCESS_COARSE_LOCATION",
      "state": 0
    },
    {
      "permission": "android.permission.ACCESS_FINE_LOCATION",
      "state": 0
    },
    {
      "permission": "android.permission.BLUETOOTH",
      "state": 0
    },
    {
      "permission": "android.permission.BLUETOOTH_ADMIN",
      "state": 0
    },
    {
      "permission": "android.permission.BLUETOOTH_CONNECT",
      "state": 0
    },
    {
      "permission": "android.permission.BLUETOOTH_SCAN",
      "state": 0
    },
    {
      "permission": "android.permission.CAMERA",
      "state": 1
    },
    {
      "permission": "android.permission.READ_EXTERNAL_STORAGE",
      "state": 0
    },
    {
      "permission": "android.permission.RECORD_AUDIO",
      "state": 0
    },
    {
      "permission": "android.permission.WRITE_EXTERNAL_STORAGE",
      "state": 0
    }
  ],

}"""

#Send request
response = requests.post(url+"afwapp/InstallPFWApp",auth=Credentials,data=Body,headers=headers)

print("dsfg" + response.text)

#User Management


##List Device

**API to List all the Devices**

This api allows you to list all the devices on the basis of GroupID and store all the listed deviceids in a variable DeviceIds

Request Body Schema

| Parameter                       | Data Type | Description                                                |
|---------------------------|-----------|------------------------------------------------------------|
| AdanceSearch              | Boolean   | Enable advanced search (true/false).                       |
| AdvSearchJobID            | String    | Advanced search job ID.                                    |
| AdvSearchValue            | List      | Values for advanced search.                                |
| SearchColumns             | List      | Columns to be searched.                                   |
| EnableDeviceGlobalSearch  | Boolean   | Enable global search for devices (true/false).            |
| ID                        | Value     | Group ID for filtering devices.                           |
| IsSearch                  | Boolean   | Indicate if it's a search operation (true/false).         |
| IsTag                     | Boolean   | Indicate if tag use for filteration (true/false).            |
| Limit                     | Integer   | Maximum number of results to retrieve.                    |
| Offset                    | Integer   | Number of results to skip before retrieving.              |
| SearchValue               | String    | Search value for device search.                           |
| SortColumn                | String    | Column for sorting results.                               |
| SortOrder                 | String    | Sort order (asc/desc) for sorting results.                |
| Columns                   | List      | Columns to include in the response.                       |
| IsIncludedBlackListed     | Boolean   | Include blacklisted devices (true/false).                |


In [ ]:
Body = f'''{{
    "AdanceSearch": false,
    "AdvSearchJobID": "",
    "AdvSearchValue": [],
    "SearchColumns": [],
    "EnableDeviceGlobalSearch": false,
    "ID":null,
    "IsSearch": false,
    "IsTag": false,
    "Limit": 100,
    "Offset": 0,
    "SearchValue": "",
    "SortColumn": "DeviceName",
    "SortOrder": "asc",
    "Columns": [
        "DeviceName",
        "DeviceModelName",
        "ConnectionStatus",
        "LastTimeStamp",
        "Battery",
        "DeviceTimeStamp",
        "ReleaseVersion"
    ],
    "IsIncludedBlackListed": false
}}'''

#Limit:maximum number of results (rows) you want to receive in the response
#Offset=0 =>start retrieving results from the beginning, Offset=10 =>skip the first 10 results and start retrieving results from the 11th one and so on


#Send request
response = requests.post(url+"devicegrid",auth=Credentials,data=Body,headers=headers)

print(json.dumps(json.loads(response.text), indent=4))
response_data = response.json()
print(response.text)

if "rows" in response_data and len(response_data["rows"]) > 0:
    DeviceIds = [device["DeviceID"] for device in response_data["rows"]]
    print("Device IDs:", DeviceIds)


##Create Group

**API to create new group**

This api allows you to create new Group

Request Body Schema

| Parameter           | Data Type | Description                              |
|---------------|-----------|------------------------------------------|
| GroupID       | String     | ID of the parent group       |
| GroupName     | String    | Name of the group.                       |


In [ ]:
GroupID = "null"  # Enter the Group ID
GroupName = "testgroup15"  #Enter Group Name

Body = f"""{{
   "GroupID": {GroupID},
   "GroupName":"{GroupName}"
}}"""
#GroupID: ID of the parent group, Group ID "null" for HOME group

#Send request
response = requests.post(url+"group",auth=Credentials,data=Body,headers=headers)

print(response.text)
CreatedGroupID = response.text

##Move Device to Group

**API to move devices to the group**

This api allows you to move all the Devices that are listed in List device section to the new group that has been created in Create Group section

Request Body Schema

| Key         | Data Type | Description                                |
|-------------|-----------|--------------------------------------------|
| DeviceId    | List     | ID of the device.                         |
| GroupId     | String     | The ID of the destination group for the device movement |


In [ ]:
Body = [
    {"DeviceId": device_id, "GroupId": CreatedGroupID} for device_id in DeviceIds
]

#Send request
response = requests.put(url + "deviceassignment", auth=Credentials, json=Body, headers=headers)
print("Move response:", response.text)

##Create Device Group Set

**API to create Device Group set**

This api allows you to create Device Group set based on GroupIDs and Users assigned these Device Group Set will have the visibility of the devices which is in this device group set.

Request Body Schema

| Parameter                             | Data Type | Description                                           |
|---------------------------------|-----------|-------------------------------------------------------|
| TemplateName                    | String    | Name of the device set template.                     |
| TemplateDescription             | String    | Description of the device set template.              |
| TemplateData                    | List of string    | List of Group IDs             |
| isAllowedNewGroupOrJobFolder    | Integer   | Allow addition of new group according to need         |

Values for `isAllowedNewGroupOrJobFolder`:

| Value | Description                                               |
|-------|-----------------------------------------------------------|
| 0     | Don't Allow Any New Groups                               |
| 1     | Automatically Allow New Groups Added Within Home Group and Allowed Sub Groups |
| 2     | Automatically Allow New Groups Added Only Within Allowed Sub Groups |


In [ ]:
deviceSet_template_name = "GroupSet16"             #Name of Device Group Set
deviceSet_template_description = "Device_GroupSet13"
deviceSet_template_data = [CreatedGroupID]           #List of Device Group ID
isAllowedNewGroupOrJobFolder = 0
#isAllowedNewGroupOrJobFolder=> 0:Don't Allow Any New Groups, 1:Automatically Allow New Groups Added Within Home Group and Allowed Sub Groups, 2:Automatically Allow New Groups Added Only Within Allowed Sub Groups

# Construct the request body
Body = {
    "TemplateName": deviceSet_template_name,
    "TemplateDescription": deviceSet_template_description,
    "TemplateData": deviceSet_template_data,
    "isAllowedNewGroupOrJobFolder": isAllowedNewGroupOrJobFolder
}

# Send POST request
response = requests.post(url + "UserGroupPermission", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == deviceSet_template_name and template["TemplateDescription"]==deviceSet_template_description:
        GroupTemplateID = template["TemplateId"]
        print(GroupTemplateID)

##Create Jobs/Profiles Folder Set

**API to create Jobs/profiles folder Set**

This api allows you to create Jobs/Profiles Folder and User assigned with this Jobs?Profiles Folder Set will have visibility of this Jobs/Profiles set only.

Request Body Schema

| Parameter                             | Data Type | Description                                           |
|---------------------------------|-----------|-------------------------------------------------------|
| TemplateName                    | String    | Name of the folder template.                         |
| TemplateDescription             | String    | Description of the folder template.                  |
| TemplateData                    | List of String    | List of Ids of Jobs/Profiles folder     |
| isAllowedNewGroupOrJobFolder    | Integer   | Allow addition of new groups or job folders according to need        |
| ReportTemplateData              | List of String    | List of Ids of Custom reports folder      |
| isForClone                      | Boolean   | Indicates if the template is for cloning purposes.  |

Values for `isAllowedNewGroupOrJobFolder`:

| Value | Description                                               |
|-------|-----------------------------------------------------------|
| 0     | Don't Allow Any New Folders                              |
| 1     | Automatically Allow New Folders Added Within Home Folder and Allowed Sub Folders |
| 2     | Automatically Allow New Folders Added Only Within Allowed Sub Folders |


In [ ]:
folder_template_name = "Create_Jobs/profiles_9"              #Name of Folder Set
folder_template_description = "Create_Jobs/profiles_9"
folder_template_data = [

]                                             #folder_template_data => List of Ids of Jobs/Profiles folder
is_allowed_new_group_or_job_folder = "0"
report_template_data = []                     #report_template_data => List of Ids of Custom reports folder
is_for_clone = False
#isAllowedNewGroupOrJobFolder=> 0:Don't Allow Any New Folders, 1:Automatically Allow New Folders Added Within Home Folder and Allowed Sub Folders, 2:Automatically Allow New Folders Added Only Within Allowed Sub Folders


Body = {
    "TemplateName": folder_template_name,
    "TemplateDescription": folder_template_description,
    "TemplateData": folder_template_data,
    "isAllowedNewGroupOrJobFolder": is_allowed_new_group_or_job_folder,
    "ReportTemplateData": report_template_data,
    "isForClone": is_for_clone
}

# Send POST request
response = requests.post(url + "UserJobPermission", auth=Credentials, json=Body, headers=headers)

print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == folder_template_name and template["TemplateDescription"]==folder_template_description:
        JobsTemplateId = template["TemplateId"]

print(JobsTemplateId)


##Create Role

**Permission IDs and Description**
<details>
<summary><b style="font-size: 18px;">Click here to take the reference and add the value in Role_template_data</summary>

| Permission ID | Description                                  |
|---------------|----------------------------------------------|
| 1             | New Group                                    |
| 2             | Rename Group                                 |
| 3             | Delete Group                                 |
| 4             | Add/Delete Group Job                         |
| 5             | Delete Device                                |
| 6             | Apply Job                                    |
| 7             | Job Queue Remove job                         |
| 8             | Blacklist Device                             |
| 9             | Remote Device                                |
| 10            | Locate Device/Location Tracking              |
| 11            | Send Message To Device                       |
| 12            | Reboot Device                                |
| 13            | Lock Device                                  |
| 14            | Wipe Device                                  |
| 15            | Lock Applications                           |
| 16            | Uninstall Applications                       |
| 17            | Applicaiton Run At Atart Up                  |
| 18            | Monitor Application                          |
| 19            | Import Preapproved Device                    |
| 20            | Auto Approve Device                          |
| 21            | Approve Device                               |
| 22            | Whitelist Device                             |
| 23            | Close Ticket                                 |
| 24            | Delete Ticket                                |
| 25            | Add/Edit/Delete Service Centers              |
| 26            | Add/Edit/Delete Fault Codes                  |
| 27            | Delete Message                               |
| 28            | Reply Message                                |
| 29            | New Job                                      |
| 30            | Delete Job                                   |
| 31            | Modify Job                                   |
| 32            | Generate Configuration Barcodes              |
| 33            | System Log Report                            |
| 34            | Device Health Report                         |
| 35            | Device Connected Report                      |
| 36            | Jobs Deployed Report                         |
| 37            | Asset Tracking Report                        |
| 38            | Activity Console                             |
| 39            | Change Password                              |
| 40            | License Management                           |
| 41            | Add User                                     |
| 42            | Edit User                                    |
| 43            | Delete User                                  |
| 44            | Edit Device Name                             |
| 45            | Edit Device Notes                            |
| 46            | Assign Group                                 |
| 47            | CallLog Tracking Report                      |
| 48            | Call Logs                                    |
| 49            | SureLock Settings                            |
| 50            | SureFox Settings                             |
| 51            | App Version Tracking Report                  |
| 53            | Clear Application Data                       |
| 54            | SureVideo Settings                           |
| 55            | Device History Report                        |
| 56            | DataUsage Report                             |
| 57            | SmsLog Device                                |
| 58            | Allow Home Group                             |
| 59            | Edit Telecom Management                      |
| 60            | Allow new device group                       |
| 61            | Allow new Folder                             |
| 62            | Hide/Unhide Application                      |
| 63            | OS Platforms Chart                           |
| 64            | Available Battery Chart                      |
| 65            | Available RAM Chart                          |
| 66            | Available Storage Chart                      |
| 67            | Last Connected Chart                         |
| 68            | Online/Offline Chart                         |
| 69            | Device SIM Status Chart                      |
| 70            | Roaming Status Chart                         |
| 71            | Unread Mail Chart                            |
| 72            | Unapproved Devices Chart                     |
| 73            | Jobs Chart                                   |
| 74            | New folder                                   |
| 75            | Upload files                                 |
| 76            | Delete File/Folder                           |
| 77            | Add Android Profile                          |
| 78            | Edit Android Profile                         |
| 79            | Delete Android Profile                       |
| 80            | Set As Default Android Profile               |
| 81            | Enroll AFW Android Profile                   |
| 82            | Add iOS Profile                              |
| 83            | Edit iOS Profile                             |
| 84            | Delete iOS Profile                           |
| 85            | Set As Default iOS Profile                   |
| 86            | Settings Android Profile                     |
| 87            | Add Windows Profile                          |
| 88            | Edit Windows Profile                         |
| 89            | Delete Windows Profile                       |
| 90            | Set As Default Windows Profile               |
| 91            | Remote Buzz                                  |
| 92            | App Store Android Add                        |
| 93            | App Store Android Edit                       |
| 94            | App Store Android Remove                     |
| 95            | App Store iOS Add                            |
| 96            | App Store iOS Edit                           |
| 97            | App Store iOS Remove                         |
| 99            | Installed Job Report                         |
| 100           | Change Password                              |
| 101           | Enable or Disable User                       |
| 102           | Add macOS Profile                            |
| 103           | Edit macOS Profile                           |
| 104           | Delete macOS Profile                         |
| 105           | Set As Default macOS Profile                |
| 106           | App Store macOS Add                         |
| 107           | App Store macOS Edit                        |
| 108           | App Store macOS Remove                      |
| 109           | Export                                      |
| 112           | Clear History                               |
| 113           | Modify History                              |
| 114           | Export History                              |
| 115           | Apply Profile                               |
| 116           | Compliance Job Report                       |
| 117           | Device Activity Report                      |
| 118           | App Store Windows Add                       |
| 119           | App Store Windows Edit                      |
| 120           | App Store Windows Remove                    |
| 121           | Custom Report                               |
| 122           | View Custom Report                          |
| 123           | Modify Custom Report                        |
| 124           | Delete Custom Report                        |
| 125           | Allow Intercom                              |
| 126           | Remote Control                              |
| 127           | File Manager                                |
| 128           | Remote Clipboard                            |
| 129           | Task Manager                                |
| 130           | Data Usage Report                           |
| 131           | Branding Info                               |
| 132           | Device Enrollment Rules                     |
| 133           | EULA Disclaimer Policy                      |
| 134           | iOS/macOS Settings                          |
| 135           | Group Assignment Rules                      |
| 136           | SAML Single Sign-On                         |
| 137           | Alert Template                              |
| 138           | Customize Toolbar                           |
| 139           | Customize Nix/SureLock                       |
| 140           | Certificate Management                       |
| 141           | Data Analytics                               |
| 142           | Miscellaneous Settings                       |
| 143           | Mobile Email Management                      |
| 144           | Account Management                           |
| 145           | Manage Subscription                          |
| 146           | Over All Compliance Chart                    |
| 147           | OS Version Compliance Chart                  |
| 148           | Rooted Compliance Chart                      |
| 149           | Connectivity Compliance Chart                |
| 150           | Sim Change Compliance Chart                 |
| 151           | Password Policy Compliance Chart            |
| 152           | Battery Compliance Chart                    |
| 153           | Application Policy Compliance Chart         |
| 154           | Siem Integration                            |
| 155           | Chrome OS Device Management                 |
| 156           | Office365 Settings                          |
| 157           | Add Office365 Profile                       |
| 158           | Edit Office365 Profile                      |
| 159           | Delete Office365 Profile                    |
| 160           | Assign to groups Office365 Profile          |
| 161           | WebHooks Settings                           |
| 162           | Email Verification                          |
| 163           | Select unapproved Android Enterprise apps   |
| 164           | Plugins                                     |
| 165           | SD Card Change Compliance Chart             |
| 166           | Firmware Updates                            |
| 167           | Efota Registration Status Chart             |
| 168           | Update Firmware                             |
| 169           | Move to Folder Android Profile              |
| 170           | Move to Folder iOS Profile                  |
| 171           | Move to Folder Windows Profile              |
| 172           | Move to Folder macOS Profile                |
| 173           | Data Analytics Dashboard                    |
| 174           | Mobile Network Connectivity Compliance Chart|
| 175           | Windows Health Attestation                  |
| 176           | Intel AMT Status                            |
| 177           | Device Storage Compliance Chart             |
| 178           | Move Jobs To Folder                         |
| 179           | Set as Default QR Code                      |
| 180           | Modify QR Code                              |
| 181           | Delete QR Code                              |
| 182           | View QR Code                                |
| 183           | Create QR Code                              |
| 185           | Mark As Read                                |
| 186           | Mark As Unread                              |
| 187           | Cleanup Inbox                               |
| 188           | Add Linux Profile                           |
| 189           | Edit Linux Profile                          |
| 190           | Delete Linux Profile                        |
| 191           | Set As Default Linux Profile               |
| 192           | Move to Folder Linux Profile               |
| 193           | NAC Integration                            |
| 194           | Rename Dashboard                           |
| 195           | Copy Dashboard                             |
| 196           | Edit Dashboard                             |
| 197           | Delete Dashboard                           |
| 198           | My Favourite Dashboard                     |
| 199           | Dashboard Edit Layout                      |
| 200           | Dashboard Add Chart                        |
| 201           | Create Dashboard                           |
| 202           | Data Usage Status                          |
| 203           | Enterprise Integrations                    |
| 204           | Scan for threats                           |
| 205           | Windows Copy Genuine Validation            |
| 206           | Mobile Threat Defence                      |
| 221           | Create Custom Column                  |
| 222           | Delete Custom Column                  |
| 223           | Move Device                           |
| 224           | Security Patch Level                  |
| 225           | Custom                                |
| 226           | Windows Update                        |
| 256           | Edit Bluetooth Name                   |
| 257           | Modify Custom Column                  |
| 258           | Edit Android Enterprise               |
| 259           | Edit Geo Fence                        |
| 260           | Edit Time Fence                       |
| 261           | Edit Network Fence                    |
| 262           | Remove Installed Profile              |
| 263           | Reset Password                        |
| 264           | System Scan                           |
| 265           | Active Kiosk Application              |
| 266           | Wi-Fi Signal Strength                 |
| 267           | Location Access                       |
| 268           | Edit Custom Column                    |
| 269           | Things Management                     |
| 270           | Unapproved View                       |
| 271           | Preapproved View                            |
| 272           | Allow Enrollment                            |
| 273           | Block Enrollment                            |
| 274           | Invite User                                 |
| 275           | Blocklisted View                            |
| 276           | Pending Delete View                         |
| 277           | Force Delete                                |
| 278           | Jobs View                                   |
| 279           | Reports View                                |
| 280           | Dashboard View                              |
| 281           | Inbox View                                  |
| 283           | Profiles View                               |
| 284           | App Store View                              |
| 285           | Enrollment View                             |
| 286           | MTD View                                    |
| 287           | Allow FileStore Permissions                 |
| 288           | Device Encryption Report                    |
| 289           | Restore                                     |
| 290           | View Custom Property                        |
| 291           | Add Custom Property                         |
| 292           | Modify Custom Property                      |
| 293           | Delete Custom Property                      |
| 294           | Import Custom Property                      |
| 295           | View Custom property                        |
| 296           | Edit Custom property                        |
| 297           | Add Default Job                             |
| 298           | Delete Default Job                          |
| 301           | Retry Pending/Failed Jobs                   |
| 303           | Export Logs                                 |
| 304           | Device Logs                                 |
| 305           | Enable FileStore - all devices              |
| 306           | Allow Remote Screen Settings                |
| 307           | Allow Remote Screen Play/Pause              |
| 308           | Allow Remote Screen Swipe to Unlock         |
| 309           | Allow Remote Screen Volume                  |
| 310           | Allow Remote Screen Search                  |
| 311           | Allow Remote Screen Screenshot              |
| 312           | Allow Remote Screen Ctrl+alt+delete        |
| 313           | Allow Remote Screen Shortcut Management    |
| 314           | Allow Remote Screen Change View            |
| 315           | Allow Remote Screen Resolution Change      |
| 316           | Allow remote New Folder                    |
| 317           | Allow remote Delete                        |
| 318           | Allow remote Download                      |
| 319           | Allow remote Upload                        |
| 320           | Allow Remote Send to Device Clipboard      |
| 321           | Allow Remote Read from Device Clipboard    |
| 322           | Allow Remote Clear from Device Clipboard   |
| 323           | Allow Remote Kill Process Task Manager     |
| 324           | Allow Remote Kill AllTask Manager          |
| 325           | App Store Linux Add                        |
| 326           | App Store Linux Edit                       |
| 327           | App Store Linux Remove                     |
| 328           | Edit Things Info                           |
| 330           | Device Enrollment Restrictions             |
| 331           | Custom Properties                           |
| 332           | Import Custom property                      |
| 333           | Edit Custom Property                        |
| 334           | Group Details                               |
| 335           | Hardware Change                             |
| 336           | Device Encryption                           |
| 337           | Device Uptime                               |
| 338           | OS Build Number                             |
| 339           | Allow Remote View                           |
| 340           | Allow Remote Touch                          |
| 342           | Service Now                                 |
| 343           | Apps                                        |
| 344           | Power Off Device                            |
| 345           | Job Revision History                        |
| 346           | Revert Job                                  |
| 347           | Compare Job                                 |
| 348           | Save as Job                                 |
| 349           | Export Logs                                 |
| 350           | Download Logs                               |
| 351           | Request Report                              |
| 352           | OEM Configuration Status                    |
| 353           | Android Management                          |
| 358           | Play For Work                               |
| 360           | Add Custom Report                           |
| 361           | Add ChromeOS Profile                        |
| 362           | Edit ChromeOS Profile                       |
| 363           | Delete ChromeOS Profile                     |
| 364           | App Store ChromeOS Add                      |
| 365           | App Store ChromeOS Edit                     |
| 366           | App Store ChromeOS Remove                   |
| 367           | Apply Jobs/Profile                          |
| 1111          | Relay Server Report                         |
| 11001         | Auto fix rules                              |
| 12001         | Application version                         |
| 12002         | OS build version                            |

</details>


**API to create Role**

This api allows you to create Role based on specific sets of permission IDs and Users assigned these roles will have the ability to perform actions corresponding to the provided permission IDs. In other words, users will be limited to actions granted by the permissions associated with their roles.

In [ ]:
Role_template_name = "licensed_india_role13"           #Name of Role Template
Role_template_description = "licensed_india_role13"       #Description of Role Template

Role_template_data = [1,2]
# Role_template_data contains IDs for specific permission.
# To add IDs, refer to the descriptions and include the corresponding values.

Body = {
    "TemplateName": Role_template_name,
    "TemplateDescription": Role_template_description,
    "TemplateData": Role_template_data
}

# Send POST request
response = requests.post(url + "UserRole", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == Role_template_name and template["TemplateDescription"]==Role_template_description:
        RoleTemplateID = template["TemplateId"]

print(RoleTemplateID)


##Create Device Grid Column set

**Grid Column IDs and Description**
<details>
<summary><b style="font-size: 18px;">Click here to take the reference and add the value in Grid_data_template</summary>


| ID  | Description                   |
|-----|-------------------------------|
| 0   | Select All                    |
| 11  | Agent Version                 |
| 42  | Android Enterprise            |
| 57  | Android ID                    |
| 3   | Battery                       |
| 82  | Battery Serial Number         |
| 83  | Battery Charge Cycles         |
| 84  | Battery Health                |
| 33  | Bluetooth Status              |
| 40  | Bluetooth Name                |
| 35  | Bluetooth SSID                |
| 10  | Cell Signal Strength(%)       |
| 71  | Cell Signal Strength(dBm)     |
| 72  | CPU Temperature               |
| 22  | CPU USAGE                     |
| 49  | CTS Verified                  |
| 21  | Data Usage                    |
| 67  | DEP Server                    |
| 61  | Device ID                     |
| 63  | Device Locale                 |
| 55  | Device Local IP Address       |
| 7   | Device Registered             |
| 9   | Device Roaming                |
| 54  | Device Time Zone              |
| 30  | Device User Name              |
| 47  | Encryption Status             |
| 26  | Enrolled                      |
| 60  | Firmware Version              |
| 59  | FOTA Registration Status      |
| 39  | Free Program Memory           |
| 38  | Free Storage Memory           |
| 81   | Google Advertising ID            |
| 31   | GPS Status                       |
| 23   | GPU Usage                        |
| 43   | Group Path                       |
| 58   | Hash Code                        |
| 66   | Hostname                         |
| 16   | IMEI                             |
| 48   | IMEI2                            |
| 73   | IMSI                             |
| 80   | Installer Package                |
| 62   | Intel® AMT Status                |
| 4    | IP Address                       |
| 20   | KNOX License                     |
| 5    | Last Connected                   |
| 6    | Last Device Time                 |
| 65   | Last Logged In User              |
| 44   | Last System Scan                 |
| 88   | Manufacturer                     |
| 77   | Media performance Class          |
| 76   | MEID                             |
| 27   | Notes                            |
| 8    | Network Operator                 |
| 29   | Network Type                     |
| 37   | OS Build Number                  |
| 12   | OS Version                       |
| 17   | Phone Number                     |
| 50   | Platform Integrity               |
| 1    | Platform/Model                   |
| 28   | Polling Mechanism                |
| 69   | Processor                        |
| 19   | Device Privilege Status          |
| 41   | Security Path Date               |
| 18   | Serial Number                    |
| 32   | Sim serial Number(ICCID)         |
| 2    | Status                           |
| 25   | Supervised                       |
| 75   | SureDefence Version              |
| 14   | SureFox Version                  |
| 36   | SureLock Settings Identifier     |
| 13   | SureLock Version                 |
| 15   | SureVideo Version                |
| 24   | Battery Temperature(?)           |
| 45   | Threats Count                    |
| 51   | Threat Protection                |
| 53   | Unknown Source Apps              |
| 34   | USB Status                       |
| 52   | USB Debugging                    |
| 46   | Wi-Fi Hotspot Status             |
| 56   | WiFi SSID                        |
| 74   | WiFi Mac Address                 |
| 68   | WiFi Signal Strength(%)          |
| 70   | WiFi Signal Strength(dBm)        |
| 85   | Latitude                         |
| 86   | Longitude                        |
| 87   | Address                          |


</details>


**API to create Device grid Column Set**

This api allows you to create Device Grid Column based on specific sets of Grid column IDs and Users assigned these device grid column set will have the ability to perform actions corresponding to the provided Grid Column Ids.

Request Body Schema

| Key                 | Data Type | Description                                |
|---------------------|-----------|--------------------------------------------|
| TemplateName        | String    | Name of the grid template.                 |
| TemplateDescription | String    | Description of the grid template.          |
| DataTemplate        | List of Integer    | Set of grid column ids |


In [ ]:
Grid_template_name = "Create_DeviceGridColumnSet_8"               #Name of Grid Template
Grid_template_description = "Create_DeviceGridColumnSet_8"          #Descriptions of Grid Template
Grid_data_template = "[ 11,3]"
# Grid_data_template contains IDs for specific Column
# To add IDs, refer to the descriptions and include the corresponding values.

# Construct the request body
Body = {
    "TemplateName": Grid_template_name,
    "TemplateDescription": Grid_template_description,
    "DataTemplate": Grid_data_template
}

# Send POST request
response = requests.post(url + "devicegridcolumn/", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == Grid_template_name and template["TemplateDescription"]==Grid_template_description and template["DataTemplate"]==Grid_data_template:
        DeviceGridTemplateID = template["TemplateId"]

print(DeviceGridTemplateID)


##Create User

**API to create Users**

This api allows you to create users with assigned variables such as GroupTemplateID, JobsTemplateId, RoleTemplateID, and DeviceGridTemplateID, which are predefined in their respective sections.Users will be limited to actions granted by the GroupTemplateID,JobsTemplateId,RoleTemplateID and DeviceGridTemplateID

Request Body Schema

| Parameter                     | Data Type | Description                                        |
|-------------------------|-----------|----------------------------------------------------|
| Password                | String    | Base64 encoded password                          |
| UserData                | Object    | User data including ID, name, email, and HideParentIfChildNotAllowed   |
| FeaturesTemplateId      | String        | The ID of the Role template to assign to the user         |
| GroupsTemplateId        | String       | The ID of the Device group set to assign to the user           |
| JobsTemplateId          | String       | The ID of the Jobs/Profiles folder to assign to the user    |
| Devicegridcolumntemplate| String       | The ID of the Device grid column set to assign to the user   |


In [ ]:
user_data = {
    "UserID": "Newuser17",
    "FirstName": "Newuser17",
    "LastName": "3",
    "Email": "Newuser4@getnada.com",
    "HideParentIfChildNotAllowed": True
}
#UserID:Enter the user name, FirstName: Enter first name of user, LastName: Enter the last name of user, Email:Enter the email

password = "Test@123"  #Enter the password
passwordEncoded = base64.b64encode(password.encode()).decode()  # Base64 encoded password
features_template_id = "673840CF-78D7-40B9-811E-0282F10B7494"                           #features_template_id:Role template id
groups_template_id = "945174F1-8ED0-49FE-BCC5-8C355508E2D1"                           #groups_template_id: Device group set id
jobs_template_id = "632C071D-0528-40C9-8489-C594EFDC86A7"                               #jobs_template_id: Jobs/Profiles folder set id
device_grid_column_template = "AE111F15-E5B0-411C-B277-2B7516C110AA"              #device_grid_column_template: Device grid column set id

#request body
Body = {
    "Password": passwordEncoded,
    "UserData": user_data,
    "FeaturesTemplateId": features_template_id,
    "GroupsTemplateId": groups_template_id,
    "JobsTemplateId": jobs_template_id,
    "Devicegridcolumntemplate": device_grid_column_template
}

# Send POST request
response = requests.post(url + "user", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))
